In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format=1, type_dates='normal'):
    t_col = type_dates.lower()
    if format == 1:
        type_date = "%d/%m/%Y"
    else:
        type_date = "%d-%m-%Y"
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column

def check_amounts(df_check, df_index, status_col, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index, status_col]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        df_new = pd.merge(df_new, filtered, how='left', left_on='sites', right_on=tw_index)
        df_new = df_new[['sites', status_col]]
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_status, df_cols, picklist_dict):

    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        columns = [i.lower() for i in picklist_dict.keys()]
        for column in set(columns): 
            value = str(df_picklist.loc[site,column])
            #print(value)
            if not value in set(picklist_dict[column]) or pd.isnull(value):
                #print(set(picklist_dict[column]))

                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'
                else:
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'

    #df = df_errors.dropna()   
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok!')
    if len(df_errors)>0:
        df = df[[df_index, df_status]]
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
    else:
        print('\nNo one Picklist Error Founded!\n')
    return df_errors

def check_picklist_3(df,df_index,df_status,picklist_dict):
    import json
    log = {}

    #picklist_dict={'Categorization by Transmission Sys':['Long-Term Mobile Sites','Macro','Non-Enterprise DAS','Repeater Sites','Public DAS Sites','Transmission']}
    for column in picklist_dict:
        df_aux=df.copy()
        #print(type(picklist_dict[column]))
        new=df_aux[column].isin(picklist_dict[column])
        #print(new)
        indexes=df.index[new == False].tolist()
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    newDict = {}
    #print(log)
    #print(json.dumps(picklist_dict,indent=2))
    for key,value in log.items():
      for val in value:
          ID=df.iloc[val][df_index]
          if ID in newDict:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID][key]=v
          else:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID] = {'status': df.iloc[val][df_status],key:v}
    #print(newDict)
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    logs.index.name='Site'
    logs = logs.dropna(how='all', axis=1).fillna('Ok!')
    return logs

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index] if str(i) not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
        return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]
        

def check_tw_doer_planned(df_tw, tw_index, doer_col,bill_col, status_col, dt_format):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date, format=dt_format)
    if not df_tw[doer_col].empty:
        df_tw[bill_col] = pd.to_datetime(df_tw[bill_col],errors='coerce', format=dt_format)
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)&\
                         (df_tw[bill_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, bill_col, doer_col]]  
    else:
        print('\nNo Errors Founded!\n')
                                                              


def check_lc_ta_dates(df,tw_index,status_col, start_date,end_date):
    filtered = df[df[start_date] > df[end_date]]
    return filtered[[tw_index,status_col, start_date,end_date]]

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']

    filtered = df_tw[[tw_index, status_tw_col]]
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
            decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
            decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
            if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
            if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna(0, inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names).fillna('')
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names ).fillna('')
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols).fillna('')

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo errors founded!')



def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')
    

Deixar para testar depois

In [ ]:
bill_cols = ["Code",\
            "Categorization by Transmission Sys",\
            "Categorization by Transmission Sys (sub-cluster)",\
            "Categorization by Site Type","Categorization by Indoor / Outdoor Site (Antennas)",\
            "Categorization by Strategic Site",\
            "Categorization by Critical Site",\
            "Site Cluster",\
            "Strategic Site Bucket",\
            "Bts_sites",\
            "CriticalSite_Beyond_10",\
            "Sites As Metered Estimated",\
            "Active Sharing Arrangement",\
            "Indoor_Site_Any_Climate_Control",\
            "Outdoor_Site_With_Dc_Power",\
            "Rfai_Date",\
            "decommissioned site",\
            "Subsequent_Active_Sharing_Arrangement",\
            "First_Active_Sharing_Deployment_Type",\
            "First_Active_Sharing_Arrangement_Start_Date",\
            "First_Active_Sharing_Arrangement_End_Date",\
            'Site Status (TIMS)']
pathnew = '/content/TowerDB_Germany_20210731.xlsx'
pathold = '/content/TowerDB_Germany_20210630.csv'
to_save = '/content/DE_TW'
find_diffs_between_files(pathold, pathnew, 'Code', bill_cols,'Site Status (TIMS)', to_save,'mix','tw', 'Sheet1')

Check TowerDB Files

In [2]:
tw_index = 'code'
tw_doer = 'date_of_equipment_removal'
tw_status = 'site status (tims)'
tw_bts = 'bts_sites'
tw_bill = 'rfai ( ready for active installation ) '
tw_wip = 'wip_sites'
tw_decom = 'decommissioned site'
tw_critical = 'critical site'

msa_index = 'code'
msa_doer = 'date_of_equipment_removal'
msa_status = 'site status (tims)'
msa_bts = 'bts_sites'
msa_bill = 'rfai ( ready for active installation ) '
msa_wip = 'wip_sites'
msa_decom = 'decommissioned site'
msa_critical = 'critical site'
bill_col = ["Code","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
             "Categorization by Site Type","Categorization by Indoor / Outdoor Site (Antennas)",\
             "Categorization by Strategic Site","Categorization by Critical Site","Site Cluster",\
             "Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10","Sites As Metered Estimated","Active Sharing Arrangement",\
             "Indoor_Site_Any_Climate_Control","Outdoor_Site_With_Dc_Power","Rfai_Date","decommissioned site",\
             "Subsequent_Active_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
             "First_Active_Sharing_Arrangement_End_Date"]

bill = ["Code","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
             "Categorization by Site Type","Categorization by Indoor / Outdoor Site (Antennas)",\
             "Categorization by Strategic Site","Categorization by Critical Site","Site Cluster",\
             "Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10","Sites As Metered Estimated","Active Sharing Arrangement",\
             "Indoor_Site_Any_Climate_Control","Outdoor_Site_With_Dc_Power","Rfai_Date","decommissioned site",\
             "Subsequent_Active_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
             "First_Active_Sharing_Arrangement_End_Date"]

In [42]:
path_towerdb = '/content/TowerDB_Germany_20210731.csv'
#teste = '/content/TowerDB_Germany_20210731 - Copia.csv'
towerdb = pd.read_csv(path_towerdb,encoding='latin')
towerdb.columns = lower_str(list(towerdb.columns))

dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]

ints = ['security class', 'postal code']
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))

towerdb = towerdb.replace(['N/A', 'n/a',"0", '-', '_',' ', np.nan,'nan'], '')
towerdb = towerdb.fillna('')


"""Read MSA File"""
path_old = '/content/TowerDB_Germany_20210630.csv'
msa = pd.read_csv(path_old, encoding='latin1')
msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

towerdb = towerdb[msa_cols]

towerdb.head(3)

,code,site name,macro region,region,province,municipality,inhabitants,address,latitude,longitude,categorization by transmission sys,categorization by transmission sys (sub-cluster),categorization by site type,categorization by indoor / outdoor site (antennas),additional site type information,categorization by inhabitants,rural/ suburban/ urban,categorization by connectivity,technology vod,pod id,energy consumption current fy,actual energy cost current fy,infrastructure ready (existing)/ to be ready (new),infrastructure to be shared by,counterpart,# of lease contracts,current annual lease fees,current energy lease fees,current annual other fees,total annual lease,(average) residual duration,maturity cluster,exco rep. avg annual lease costs,total energy cost currrent fy (energy provider + ll),vod (y/n),deutsche telekom,annual fee per tenant mno1,annual energy fee mno1,annual maintenance fee mno1,other services fee mno1,...,categorization by shared usage (contractual),categorization by shared usage (pop count),security class,categorization by sites with single/multiple tower(s),categorization by special site/plc,categorization by chimney-location,categorization by antenna constraint,categorization by 5g roll-out site,categorization by authority site,categorization by transport-sammler,categorization by strategic site,categorization by dual use site,categorization by sublease right,gbt reserve space,state,postal code,tower height,floor space,ground register .1,ground register .2,ground register .3,categorization by critical site,site cluster,infrastructure to be dismantled by,decommissioned site,strategic site bucket,bts_sites,criticalsite_beyond_10,sites as metered estimated,active sharing arrangement,subsequent_active_sharing_arrangement,first_active_sharing_deployment_type,first_active_sharing_arrangement_start_date,first_active_sharing_arrangement_end_date,indoor_site_any_climate_control,outdoor_site_with_dc_power,rfai_date,cluster for non-enterprise das sites,site status (tims),date_of_equipment_removal
0,0001 B,Neustadt,East,B,,Berlin,3550948,Kandeler Weg 1,"52,5419278","13,1852444",Macro,Standard,RTT,Outdoor,,Urban,URBAN,Microwave,2G,0001 B,6820.729000,0,,,0,1,8980.7825,0,,0,9.583333,<= 10 years,8980.7825,1516.903597,,0,0.0,0,,,...,Stand-alone,Co-located,1,Single Tower,Standard Site,Other,,no,no,no,No,no,no,,Berlin,13583,,,Spandau,68/4,5/6,No,,,,Non Strategic,No,Non Critical,Estimated Model,No Active Sharing,,,,,,AC,,,In Service,
1,0001 D,D-Innenstadt,West,D,,Düsseldorf,611356,Oststr. 86,"51,22286","6,78708",Macro,Standard,RTT,Outdoor,,Urban,URBAN,Microwave,2G,0001 D,6800.803807,0,,,0,1,5510.0000,0,,0,5.583333,<= 10 years,5510.0000,1784.930952,,0,0.0,0,,,...,Stand-alone,Stand-alone,1,Single Tower,Standard Site,Other,,no,no,no,Yes,no,no,,North Rhine-Westphalia,40210,,,Düsseldorf,"357,358",4,No,,,,Yes - 0-5%,No,Non Critical,Estimated Model,No Active Sharing,,,,,,AC,,,In Service,
2,0001 M,M-Innenstadt,South,M,,München,1452824,Arnulfstr. 2,"48,14165278","11,56010278",Transmission,Transmission,RTT,Indoor,,Urban,URBAN,Fibre,Transmission only,0001 M,110204.480000,0,,,0,1,85000.0000,0,,0,7.166667,<= 10 years,85000.0000,16377.251800,,0,0.0,0,,,...,Shared,Co-located,4,Single Tower,Standard Site,Other,,no,yes,yes,No,no,no,,Bavaria,80335,,,Max-Vorstadt,6781,,Yes,,,,Non Strategic,No,Within 10%,Estimated Model,No Active Sharing,,,,,Yes; Indoor Air Conditioning,,,,In Service,


In [ ]:
"""First Step Check All columns received"""
#check billing columns
df_col_missing = check_columns_received(towerdb, msa_cols)
df_col_missing
#tem error Falta Columns de Billing trigger

In [9]:
towerdb.head(3)

,code,site name,macro region,region,province,municipality,inhabitants,address,latitude,longitude,categorization by transmission sys,categorization by transmission sys (sub-cluster),categorization by site type,categorization by indoor / outdoor site (antennas),additional site type information,categorization by inhabitants,rural/ suburban/ urban,categorization by connectivity,technology vod,pod id,energy consumption current fy,actual energy cost current fy,infrastructure ready (existing)/ to be ready (new),infrastructure to be shared by,counterpart,# of lease contracts,current annual lease fees,current energy lease fees,current annual other fees,total annual lease,(average) residual duration,maturity cluster,exco rep. avg annual lease costs,total energy cost currrent fy (energy provider + ll),vod (y/n),deutsche telekom,annual fee per tenant mno1,annual energy fee mno1,annual maintenance fee mno1,other services fee mno1,...,categorization by shared usage (contractual),categorization by shared usage (pop count),security class,categorization by sites with single/multiple tower(s),categorization by special site/plc,categorization by chimney-location,categorization by antenna constraint,categorization by 5g roll-out site,categorization by authority site,categorization by transport-sammler,categorization by strategic site,categorization by dual use site,categorization by sublease right,gbt reserve space,state,postal code,tower height,floor space,ground register .1,ground register .2,ground register .3,categorization by critical site,site cluster,infrastructure to be dismantled by,decommissioned site,strategic site bucket,bts_sites,criticalsite_beyond_10,sites as metered estimated,active sharing arrangement,subsequent_active_sharing_arrangement,first_active_sharing_deployment_type,first_active_sharing_arrangement_start_date,first_active_sharing_arrangement_end_date,indoor_site_any_climate_control,outdoor_site_with_dc_power,rfai_date,cluster for non-enterprise das sites,site status (tims),date_of_equipment_removal
0,0001 B,Neustadt,East,B,NaN,Berlin,3550948,Kandeler Weg 1,"52,5419278","13,1852444",Macro,Standard,RTT,Outdoor,NaN,Urban,URBAN,Microwave,2G,0001 B,6820.729000,0,NaN,NaN,0,1,8980.7825,0,NaN,0,9.583333,<= 10 years,8980.7825,1516.903597,NaN,0,0.0,0,NaN,NaN,...,Stand-alone,Co-located,1.0,Single Tower,Standard Site,Other,NaN,no,no,no,No,no,no,NaN,Berlin,13583.0,NaN,NaN,Spandau,68/4,5/6,No,NaN,NaN,NaN,Non Strategic,No,Non Critical,Estimated Model,No Active Sharing,NaN,NaN,NaN,NaN,NaN,AC,NaN,NaN,In Service,NaN
1,0001 D,D-Innenstadt,West,D,NaN,Düsseldorf,611356,Oststr. 86,"51,22286","6,78708",Macro,Standard,RTT,Outdoor,NaN,Urban,URBAN,Microwave,2G,0001 D,6800.803807,0,NaN,NaN,0,1,5510.0000,0,NaN,0,5.583333,<= 10 years,5510.0000,1784.930952,NaN,0,0.0,0,NaN,NaN,...,Stand-alone,Stand-alone,1.0,Single Tower,Standard Site,Other,NaN,no,no,no,Yes,no,no,NaN,North Rhine-Westphalia,40210.0,NaN,NaN,Düsseldorf,"357,358",4,No,NaN,NaN,NaN,Yes - 0-5%,No,Non Critical,Estimated Model,No Active Sharing,NaN,NaN,NaN,NaN,NaN,AC,NaN,NaN,In Service,NaN
2,0001 M,M-Innenstadt,South,M,NaN,München,1452824,Arnulfstr. 2,"48,14165278","11,56010278",Transmission,Transmission,RTT,Indoor,NaN,Urban,URBAN,Fibre,Transmission only,0001 M,110204.480000,0,NaN,NaN,0,1,85000.0000,0,NaN,0,7.166667,<= 10 years,85000.0000,16377.251800,NaN,0,0.0,0,NaN,NaN,...,Shared,Co-located,4.0,Single Tower,Standard Site,Other,NaN,no,yes,yes,No,no,no,NaN,Bavaria,80335.0,NaN,NaN,Max-Vorstadt,6781,NaN,Yes,NaN,NaN,NaN,Non Strategic,No,Within 10%,Estimated Model,No Active Sharing,NaN,NaN,NaN,NaN,Yes; Indoor Air Conditioning,NaN,NaN,NaN,In Service,NaN


In [ ]:

sites = [i for i in towerdb[towerdb['climate control (yes/no)']=='Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

sites = [i for i in towerdb[towerdb['climate control (yes/no)']=='Yes; Indoor Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

towerdb = towerdb.fillna('')

In [17]:
"""Second Check Date Format dd-mm-YYYY"""
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            df_de = df_de[~(df_de[status_col]=='In Service')]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

dates_cols = ['Code', 'Date_Of_Equipment_Removal']
dates_cols = lower_str(dates_cols)
#Check dates of column 'Date_Of_Equipment_Removal' Pg 7
#check_date_columns(df, df_index,status_col,columns, format)
dftw_dates_errors = check_date_columns(towerdb, tw_index, tw_status, dates_cols, 2) 
# Errors msg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [18]:
dftw_dates_errors

,code,site status (tims),date_of_equipment_removal


In [28]:
"""Check all picklist Columns"""
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors = df_errors.dropna(how='all', axis=1)
    
    if not df_errors.empty:
        #df = df[[df_index, df_status]]
        #df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors[df_index] = df_picklist[df_index]
        df_errors[df_status] = df_picklist[df_status]
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
        return df_errors
    else:
        print('\nNo one Picklist Error Founded!\n')

picklist_col = ["Code",'site status (tims)','Categorization by Transmission Sys','Categorization by Transmission Sys (sub-cluster)',\
            'Categorization by Site Type', 'Categorization by Indoor / Outdoor Site (Antennas)',\
            'Categorization by Strategic Site','Categorization by Critical Site', 'Strategic Site Bucket', 'Bts_sites',\
            'CriticalSite_Beyond_10','decommissioned site', 'Sites As Metered Estimated',\
            'Active Sharing Arrangement', 'Indoor_Site_Any_Climate_Control', 'Outdoor_Site_With_Dc_Power']
picklist_col = lower_str(picklist_col)
picklist = {
    'categorization by transmission sys': ['Macro','Public DAS Sites','Repeater Sites','Transmission', 'Long-Term Mobile Sites', 'Non-Enterprise DAS'],
    'categorization by transmission sys (sub-cluster)': ['Core', 'DAS', 'Long-term Mobile', 'Non-Enterprise DAS', 'Repeater', 'Standard', 'Transmission'],
    'categorization by site type':['DAS','GBT','RTT', 'Non-Enterprise DAS'],
    'categorization by indoor / outdoor site (antennas)': ['Outdoor', 'Indoor'],
    'categorization by strategic site':['Yes','No'],
    'categorization by critical site':['Yes','No'],
    'strategic site bucket': ['Yes - 0-5%','Non Strategic'],
    'bts_sites': ['Yes','No'],
    'criticalsite_beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'decommissioned site': ['Yes', 'No'],
    'sites as metered estimated': ['Estimated Model','Metered Model'],
    'active sharing arrangement': ['No Active Sharing'],
    'indoor_site_any_climate_control': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'outdoor_site_with_dc_power':['AC','DC','AC & DC','No Power']}
actives = towerdb[towerdb[tw_status]=='In Service']
df_picklist_errors = check_picklist_v1(actives,'code','site status (tims)',picklist_col, picklist)
df_picklist_errors

,code,site status (tims),indoor_site_any_climate_control,outdoor_site_with_dc_power,decommissioned site
0,0001 B,In Service,Blank Value,NaN,Blank Value
1,0001 D,In Service,Blank Value,NaN,Blank Value
2,0001 M,In Service,NaN,Blank Value,Blank Value
3,0001 O,In Service,NaN,Blank Value,Blank Value
4,0001 S,In Service,NaN,Blank Value,Blank Value
...,...,...,...,...,...
19747,Y155 M,In Service,Blank Value,NaN,Blank Value
19748,Y156 M,In Service,NaN,Blank Value,Blank Value
19749,Y157 M,In Service,NaN,Blank Value,Blank Value
19750,Y158 M,In Service,NaN,Blank Value,Blank Value


In [30]:
actives['decommissioned site'].value_counts()

    19752
Name: decommissioned site, dtype: int64

In [88]:
#Read UIP File
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel('/content/UserInput_Germany_20210731_InMonth.xlsx',sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [str(i) for i in msa[msa[msa_status]=='In Service']['code']]
tw_sites = [str(i) for i in towerdb[towerdb[tw_status]=='In Service']['code']]
uip_sites = [str(i) for i in uip['Site_ID']]


In [90]:
print([i for i in tw_sites if i not in uip_sites])

[]


In [52]:
"""Eigth Check for BTS sites"""
def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    filtered = df_tw[[tw_index, status_col, bts_tw_columns]]
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    if not df.empty:
        df = pd.merge(df, filtered, how='left', left_on=['New Sites'], right_on=tw_index)
        df = df[['New Sites', status_col, bts_tw_columns]]
        return df
    else: 
        print('\nNo errors founded!\n')

# Check for BTS sites
df_bts_out_tw = check_bts(towerdb,tw_bts,tw_index,tw_status,msa,msa_bts,msa_index)
df_bts_out_tw

,New Sites,site status (tims),bts_sites
0,0861 W,In Service,Yes
1,3207 O,In Service,Yes
2,5331 B,In Service,Yes
3,5866 O,In Service,Yes
4,6214 S,In Service,Yes
5,6282 O,In Service,Yes
6,7963 H,In Service,Yes
7,B194 M,In Service,Yes


In [ ]:
msa.columns.to_list()

In [ ]:
"""Falta column billing trigger(Rfai_Date) p fazer o check"""
def check_wip(df_tw,tw_index,tw_status, tw_wip, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[tw_wip]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[tw_wip]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index,tw_status,tw_wip, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    if not(len(wip_out_tw_list)==0 or tw_wip_site_bts_flagged.empty):
        return wip_out_tw_list, tw_wip_site_bts_flagged

wip_out_tw_list, df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_status, tw_wip,tw_bts, msa, msa_index, msa_wip)
# No Errors

In [72]:
sites = [i for i in towerdb[towerdb['decommissioned site']=='Yes'][tw_index]]
if len(sites)>0:   
    towerdb[towerdb[tw_index].isin(sites)&(towerdb[tw_doer]=="")]
else:
    print('vazio')

vazio


In [73]:
"""Nineth - Check fopr decomissioned sites"""
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    sites = [i for i in df[df[decom_col]=='Yes'][df_index]]
    if len(sites)>0:
        filtered = df[(df[df_index].isin(sites))&(df[doer_col]=="")]
        return filtered[[df_index,status_col, decom_col, doer_col]]
    else:
        print('\nNo errors Founded!\n')
#Validate Decomissioned Sites
df_decom_errors = check_decommissioned(towerdb,tw_index, tw_status, 'decommissioned site', tw_doer)
df_decom_errors
#None errors


No errors Founded!



In [81]:
"""Tenth - Check Date Of Equipement Removed date for in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    dates = [i for i in df_tw[date_col] if i!= '']

    if len(dates)>0:
        if t == 'doer':
            filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
            return filtered
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[df_tw[date_col]=='']
            return filtered
    else:
        print('\nNo errors founded!\n')

df_doer_errors = check_tw_bill_doer(towerdb, tw_index, tw_doer , tw_status,'In Service', 'doer')
df_doer_errors
# None Errors

0

No errors founded!



In [82]:
"""Eleventh - Check Month-on-Month towerdb sites"""
def check_mom_bts(df_tw, tw_index,status_col,df_bts, df_msa, msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[df_bts]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    if not filtered.empty:
        return filtered[[tw_index,status_col, df_bts]]    
    else:
        print('\nNo Errors Founded!\n')
df_mom_errors = check_mom_bts(towerdb,tw_index, tw_status, tw_bts, msa,msa_index, msa_bts)
# None Errors


No Errors Founded!



In [85]:
"""Twelveth - Match UIS sites with Towerdb Sites """
def check_uip_tw(df_tw,tw_index, status_tw_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    #Check BTS sites
        #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]                            
    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis,  bts_sites_out_uip


uis_sites_not_in_towerdb, in_service_not_in_uis,  bts_sites_out_uip = check_uip_tw(towerdb, tw_index, tw_status, tw_bts, \
                                                                                             tw_critical, uip, uip_sites)
uis_sites_not_in_towerdb
#None Errors

,UIS In Month not active in TowerDB!,site status (tims)


Commercial

In [5]:
act = '/content/UserInput_Germany_20210731_InMonth.xlsx'
old = '/content/UserInput_Germany_20210630.xlsx'
cols_ordered = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type','Input_Value_actual', 'Input_Value_before',\
                'Equal Values','Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type', 'Description/Instruction', 'Frequency of Update']
name = ['Charge_Type','Sub_charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value', 'Description/Instruction', 'Frequency of Update'] 
col = ['Input_Value']               
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna(0, inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names).fillna('')
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names ).fillna('')
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols).fillna('')

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo errors founded!')

df_comm = check_commercial(act, old, col, name, merge_cols, cols_ordered)
df_comm

,Charge_Type,Sub_charge_Type,Param1,Param2,Data_Type,Input_Value_actual,Input_Value_before,Equal Values,Description/Instruction,Frequency of Update
41,BASE SERVICE CHARGE,LEGACY NON-ENTERPRISE DAS SITES,CAT 1,,NUMBER,2020,,No,- Annual value as per the MSA\n- To be edited ...,Only in case of change in MSA/Side letter
42,BASE SERVICE CHARGE,LEGACY NON-ENTERPRISE DAS SITES,CAT 2,,NUMBER,8080,,No,- Annual value as per the MSA\n- To be edited ...,Only in case of change in MSA/Side letter
43,BASE SERVICE CHARGE,LEGACY NON-ENTERPRISE DAS SITES,CAT 3,,NUMBER,30300,,No,- Annual value as per the MSA\n- To be edited ...,Only in case of change in MSA/Side letter
44,BASE SERVICE CHARGE,LEGACY NON-ENTERPRISE DAS SITES,CAT 4,,NUMBER,80800,,No,- Annual value as per the MSA\n- To be edited ...,Only in case of change in MSA/Side letter
45,LEGACY NON-ENTERPRISE DAS SITE UPGRADE CHARGIN...,APPLICABLE PROJECT SERVICE CHARGE FOR SITE UPG...,,,PERCENTAGE(NUMBER ONLY),18,,No,- Annual value as per the MSA\n- To be edited ...,Only in case of change in MSA/Side letter


Generate xlsx Log with invalid or incorrect values founded in TowerDB File

In [ ]:
df_list = [ df2, df_picklist_errors, df_on_air_sites_errors, df_commercial_diffs]
sheetnames = ['Dates Invalid Format', 'Picklist Invalid Value', 'On air sites with Blank Values', "Commercial Different values"]

path = '/content/towerdb_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

TA Input File Check

In [20]:
import pandas as pd
#Read TA Input File
path_ta_input = '/content/TA_Input_Germany_20210731.csv'

ta_col = ['Code','Tenant Agreement','Tenant','Tenant Cluster','MNO Cluster','Tenant (ID)','Starting date','Expiring date',\
          ' Annual Fee per Tenant ','Overlapping days in FY20',' Current Annual Fee per Tenant ',\
          'Annual Energy Fee from 3rd Party Tenants',' OTP Amount ',' Renewal option ',' Residual duration ',\
          'Maturity Cluster','Contract Scope','Site Scope','Contract Migration Scope 14.05.2020','Comment']

ta = pd.read_csv(path_ta_input, encoding='latin')

ta = ta[ta_col]
ta = ta.replace(['N/A', 'n/a',"0", '-', '_',' ', np.nan,'nan'], '')
ta.head(2)

,Code,Tenant Agreement,Tenant,Tenant Cluster,MNO Cluster,Tenant (ID),Starting date,Expiring date,Annual Fee per Tenant,Overlapping days in FY20,Current Annual Fee per Tenant,Annual Energy Fee from 3rd Party Tenants,OTP Amount,Renewal option,Residual duration,Maturity Cluster,Contract Scope,Site Scope,Contract Migration Scope 14.05.2020,Comment
0,0001 M,2400004762/2500003831,Tangens GmbH,OTMO,OTMO,386,14/10/2002,31/12/2050,0.000,365,0.000,0,,No contractual end date,0.0,exclude,In scope,in scope,,
1,0002 H,2400004299/2500006639,Telefónica Germany GmbH & Co. OHG,TEF,MNO,400000577,01/01/2017,31/12/2050,1461.672,365,1461.672,0,,No contractual end date,0.0,exclude,In scope,in scope,In scope,


In [ ]:
ta['Expiring date'].value_counts()

In [ ]:

#Check date format(dd/mm/YYYY)
cols_date = ['Code','Starting date', 'Expiring date']
#df_dates_erros = check_date_columns(ta, cols_date, 'DE', 'TA')

#Check Amount values
cols = ['Code',' Annual Fee per Tenant ', ' Current Annual Fee per Tenant ']                
df = check_amounts(ta, 'Code',cols,'.')
df

In [ ]:
ta_bills = ['Code','Tenant','Starting date', 'Expiring date']
path_ta_new = '/content/TA_Input_Germany_20210731.csv'
path_ta_old = '/content/TA_Input_Germany_20210630.csv'
ta_save = '/content/TA_DE'
find_diffs_between_files(path_ta_old, path_ta_new, 'Code', ta_bills,'',ta_save,'csv','ta')

In [59]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col] + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        """if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()"""

        return newRows, droppedRows, new_copy
        """if kind=='ta':
            #new_sites['site_code'] = [i.endswith('MEO') or i.endswith('NOS')]
            new_copy['sites_code'] = [re.sub('MEO$|NOS$', '', str(x)) for x in new_copy['sites']]
            new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]"""

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='latin')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

ta_bills = ['Code','Tenant','Starting date', 'Expiring date']
path_ta_new = '/content/TA_Input_Germany_20210731.csv'
path_ta_old = '/content/TA_Input_Germany_20210630.csv'
ta_save = '/content/TA_DE'
old = '20210630.csv'
new = '20210731.csv'
"""(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(path_ta_old, path_ta_new, 'Tenant Agreement', ta_bills, ta_save, old, new,type_file='csv', status_col='',\
                         kind='ta', kind_col='Tenant')


New Rows:     []
Dropped Rows: ['/2500006069Telefónica Germany GmbH & Co. OHG', '2400003988/2500006284Landratsamt Niederschlesischer', '/2500006914EWE NETZ GmbH', '/2400000811Deutsche Funkturm GmbH', '/2400000810Deutsche Funkturm GmbH', '/2400000812Telekom Deutschland']

Done.



LC Input File Check

In [ ]:
#Read TA Input File
lc_cols = ['Lease Contract ID','Code','Counterpart','Counterpart Cluster','Counterpart (ID)','Starting date','Expiring date',\
           ' Annual lease fees ','Overlapping days in FY20',' Current annual lease fees ',' OTP Amount ',\
           'Renewal option','(Average) residual duration','Maturity Cluster','Site Scope','Contract scope','MNO Cluster','reason for scope',\
           'Adjustment','+/-','Contract Migration Scope 14.05.2020']

path_lc_input = '/content/LC_Input_Germany_20210731.csv'
lc = pd.read_csv(path_lc_input, encoding='latin')

lc = lc[lc_cols]
lc = lc.replace(['N/A', 'n/a',"0", '-', '_',' ', np.nan,'nan'], '')
lc.head(1)



In [ ]:
date_cols=['Code','Starting date', 'Expiring date']
#(df, df_index, columns, format)
df_dates_erros = check_date_columns(lc, 'Code', date_cols, 2)

lc_cols = ['Code', ' Annual lease fees ']
df = check_amounts(lc, 'Code',lc_cols, '.')

In [ ]:
lc_bills = ['Code',' Annual lease fees ','Starting date', 'Expiring date']
path_lc_new = '/content/LC_Input_Germany_20210731.csv'
path_lc_old = '/content/LC_Input_Germany_20210630.csv'
lc_save = '/content/LC_DE'
find_diffs_between_files(path_lc_old, path_lc_new, 'Code', lc_bills,'',lc_save,'csv','lc')

In [58]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col] + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        """if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()"""

        return newRows, droppedRows, new_copy
        """if kind=='ta':
            #new_sites['site_code'] = [i.endswith('MEO') or i.endswith('NOS')]
            new_copy['sites_code'] = [re.sub('MEO$|NOS$', '', str(x)) for x in new_copy['sites']]
            new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]"""

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='latin')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

lc_bills = ['Lease Contract ID',' Annual lease fees ','Starting date', 'Expiring date']
path_lc_new = '/content/LC_Input_Germany_20210731.csv'
path_lc_old = '/content/LC_Input_Germany_20210630.csv'
lc_save = '/content/LC_DE'
old = '20210630.csv'
new = '20210731.csv'
"""(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(path_lc_old, path_lc_new, 'Lease Contract ID', lc_bills, lc_save, old, new,type_file='csv', status_col='',\
                         kind='ta', kind_col='Counterpart')


New Rows:     ['1100013647/1100023345Marien-Krankenhaus', '1100004626/1100021970BAVARIA Hausverwaltung GmbH', '1500000342/1500000376Telefónica Germany GmbH & Co. OHG', '1100024422/GSW Immobilien AG', '1100011133/1100033881Hörmann Holding GmbH&Co.KG', '1100024327/Ludwigsburg Eduard Spranger', '1100024202/Wohnungsbau- u. Siedlungsges.', '1100024420/GSW Immobilien AG', '1100023533/1100006177Harz Energie GmbH&Co.KG', '1600000638/1600000014BVVG Bodenverwertungs-u.verwaltungs', '1100022907/1100041166e.discom Telekommunikations GmbH', '1500000337/1500000387Telefónica Germany GmbH & Co. OHG', '1100024417/NOVEC GmbH', '1100009895/1100025070R+V Lebensversichrung AG', '1100024398/Herbert Baltes', '1100009450/1100034666Löwenstein Invest Vier GmbH', '1100024305/Kaufhof Düsseldorf Carschhaus GmbH', '1100024059/Guenter Schneider', '1100024389/Magdeburger Getreide GmbH', '1600000823/DFMG Deutsche Funkturm GmbH', '1100024419/BauBeCon Wohnwert GmbH', '1100021320/Aenne Pitz', '1100002683/1100011790Ilona

In [54]:
d = pd.read_csv(path_lc_old, encoding='latin')
d

,Lease Contract ID,Code,Counterpart,Counterpart Cluster,Counterpart (ID),Starting date,Expiring date,Annual lease fees,Overlapping days in FY20,Current annual lease fees,OTP Amount,Renewal option,(Average) residual duration,Maturity Cluster,Site Scope,Contract scope,MNO Cluster,reason for scope,Adjustment,+/-,Contract Migration Scope 14.05.2020
0,1100002245/1100000642,0001 B,Gemeinn.Wohnungsbau AG Berlin,OTMO,800001944.0,05/12/1990,31/12/2030,8980.782500,365,8980.782500,NaN,On-going,9.666667,<= 10 years,in scope,In Scope,OTMO,"condition, or indicator for one-time payment o...",NaN,NaN,In scope
1,1100006016/1100024912,0001 D,Kilrush S.à.r.l. v.d. Tectareal Property,OTMO,800020546.0,01/01/2007,31/12/2026,5510.000000,365,5510.000000,NaN,Active until termination date,5.666667,<= 10 years,in scope,In Scope,OTMO,NaN,NaN,NaN,NaN
2,1100006591/1100020007,0001 M,Inka Arnulfstraße GmbH & Co. KG,OTMO,800046411.0,01/10/1990,31/07/2028,85000.000000,365,85000.000000,NaN,On-going,7.250000,<= 10 years,in scope,In Scope,OTMO,NaN,NaN,NaN,In scope
3,1600000237/1600001056,0001 O,Bundesanstalt für Immobilienaufgaben,OTMO,800019339.0,28/06/2013,28/06/2043,329.752066,365,329.752066,NaN,No contractual end date,22.083333,<= 25 years,in scope,In Scope,OTMO,NaN,NaN,NaN,NaN
4,1100003041/1100028651,0001 S,Ver.Freunde der Uni Stuttgart,OTMO,800001917.0,23/10/1990,31/10/2022,600.000000,365,600.000000,NaN,Renewed,1.500000,<= 5 years,in scope,In Scope,OTMO,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22016,1100022312/1100022480,Y157 M,Bundesanstalt für Immobilienaufgaben,OTMO,800004417.0,01/01/2006,01/01/2036,0.000000,365,0.000000,NaN,No contractual end date,14.666667,<= 15 years,in scope,In Scope,OTMO,NaN,NaN,NaN,NaN
22017,1100022313/1100019807,Y158 M,WorldCell Inc.,OTMO,800018907.0,01/12/2005,31/05/2022,0.000000,365,0.000000,NaN,No contractual end date,1.083333,<= 5 years,in scope,In Scope,OTMO,NaN,NaN,NaN,In scope
22018,1100022314/1100022483,Y158 M,Bundesanstalt für Immobilienaufgaben,OTMO,800004417.0,01/01/2006,01/01/2036,0.000000,365,0.000000,NaN,No contractual end date,14.666667,<= 15 years,in scope,In Scope,OTMO,NaN,NaN,NaN,In scope
22019,1100022315/1100019811,Y159 M,WorldCell Inc.,OTMO,800018907.0,01/12/2005,31/05/2022,0.000000,365,0.000000,NaN,No contractual end date,1.083333,<= 5 years,in scope,In Scope,OTMO,NaN,NaN,NaN,NaN


In [47]:
lc[['Counterpart']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22055 entries, 0 to 22054
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Counterpart  22055 non-null  object
dtypes: object(1)
memory usage: 172.4+ KB
